In [1]:
!pip install datasets
!sudo apt-get install swig
!sudo pip install jamspell
!wget https://github.com/bakwc/JamSpell-models/raw/master/en.tar.gz
!tar -xvf en.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 4.8 MB/s 
     |████████████████████████████████| 213 kB 63.5 MB/s 
     |████████████████████████████████| 132 kB 63.2 MB/s 
     |████████████████████████████████| 182 kB 46.8 MB/s 
     |████████████████████████████████| 127 kB 56.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import altair as alt



import altair as alt
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import jamspell
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV 

import datasets
from datasets import load_dataset

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [3]:
# cargamos el corpus de texto
dataset = datasets.load_dataset('tweet_eval', 'sentiment')

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45615 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12284 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# Convertimos los datasets en formato dataframe
train_df = pd.DataFrame(dataset['train'])
val_df = pd.DataFrame(dataset['validation'])
test_df = pd.DataFrame(dataset['test'])

In [5]:
# Na check.
train_df.text.isnull().sum()

0

In [6]:
y = train_df.label.copy()

y[y == 0] = 'Negative'
y[y == 1] = 'Neutral'
y[y == 2] = 'Positive'


# Labels desbalanceadas.
counts = y.value_counts(normalize=True).reset_index()
counts.columns = ['target', 'ratio']

alt.Chart(counts).mark_bar().encode(
    x=alt.X('target:N', title='Categorías'),
    y=alt.Y('ratio:Q', scale=alt.Scale(domain=(0, 1)), title='Ratio sobre total'),
    tooltip=['target', 'ratio']
).properties(width=600, height=200)

alt.Chart(...)

In [7]:
corrector = jamspell.TSpellCorrector()
corrector.LoadLangModel('en.bin')
lemmatizer= WordNetLemmatizer()
punct = ''.join([i for i in string.punctuation if i not in [',','?','!','.']])

In [8]:
# Elaboramos la funcion de limpieza de texto

def mid_clean(x):
  # normalizacion del texto es minusculas.
  x = x.lower()
  x = x.encode('ascii', 'ignore').decode()
  # Removemos las url.
  x = re.sub(r"http\S+", ' ', x)
  # Removemos las palabras que empiezan por @.
  x = re.sub(r'@[\w]+', '', x)
  # Removemos los signos de puntuacion.
  x = re.sub('[^\w\s]', '', x)
  # Spell check
  x = corrector.FixFragment(x)
  # Removemos palabras con menos de dos caracteres.
  x = ' '.join([w for w in x.split() if len(w)>2])
  return x

In [9]:
example = train_df.text[45610] + " #hashtag"

In [11]:
mid_clean(example)

'amazing have the beautiful lady gaga the show tonight love you girl britney hashtag'

In [12]:
train_df['text_c2'] = train_df['text'].apply(mid_clean)

In [13]:
# Vectorizamos las palabras de
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

train_vectors = vectorizer.fit_transform(train_df['text_c2'])

In [14]:
train_vectors = vectorizer.fit_transform(train_df['text_c2'])

In [15]:
# Calculamos los pesos de cada clase para darselos al modelo
class_weights = (1 - (train_df['label'].value_counts().sort_index() / len(train_df))).values
class_weights

array([0.8445029 , 0.54679382, 0.60870328])

In [16]:
weights = {}

for idx, i in enumerate(class_weights):
  weights[idx] = i

weights

{0: 0.8445029047462458, 1: 0.5467938178230844, 2: 0.6087032774306698}

In [17]:
# Cargamos el modelo y le proporcionamos los pesos
svm = SVC(class_weight=weights)

In [18]:
# Creamos un grid de parametros para el GridSearch
param_grid = {'C': [0.001,0.1,1], 'gamma': [1,0.1,0.001], 'kernel': ['linear', 'rbf']}

In [19]:
# Inicializamos gridsearch
grid = GridSearchCV(svm, param_grid)

In [20]:
# Procedemos a entrenarlo
svm2 = grid.fit(train_vectors, train_df['label'].values.ravel())

In [21]:
svm2

GridSearchCV(estimator=SVC(class_weight={0: 0.8445029047462458,
                                         1: 0.5467938178230844,
                                         2: 0.6087032774306698}),
             param_grid={'C': [0.001, 0.1, 1], 'gamma': [1, 0.1, 0.001],
                         'kernel': ['linear', 'rbf']})

In [22]:
svm2.best_params_

{'C': 1, 'gamma': 1, 'kernel': 'linear'}

In [23]:
test_vectors = vectorizer.transform(val_df['text'])

In [24]:
predictionsc2 = grid.predict(test_vectors)

In [25]:
from sklearn.metrics import classification_report
report = classification_report(val_df['label'], predictionsc2, output_dict=True)

In [26]:
report

{'0': {'precision': 0.5607843137254902,
  'recall': 0.4583333333333333,
  'f1-score': 0.5044091710758377,
  'support': 312},
 '1': {'precision': 0.6443148688046647,
  'recall': 0.762945914844649,
  'f1-score': 0.6986301369863013,
  'support': 869},
 '2': {'precision': 0.7597765363128491,
  'recall': 0.6642246642246642,
  'f1-score': 0.7087947882736155,
  'support': 819},
 'accuracy': 0.675,
 'macro avg': {'precision': 0.654958572947668,
  'recall': 0.6285013041342156,
  'f1-score': 0.6372780321119181,
  'support': 2000},
 'weighted avg': {'precision': 0.678565655056915,
  'recall': 0.675,
  'f1-score': 0.6724940910064241,
  'support': 2000}}